<a href="https://colab.research.google.com/github/yw7vvAW611/CS269GroupProject/blob/main/AdversarialAttackFinBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install textattack[tensorflow] > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.models.wrappers import ModelWrapper

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:36<00:00, 13.2MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpsl2qodsx.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


# Section 1 FinBERT Against FPB Testing Dataset

In [ ]:
import pandas
import numpy as np


fpb100_df = pandas.read_csv("drive/MyDrive/test.csv", sep="\t")

fpb100_df.head(5)

fpb100_array = np.array(fpb100_df)
fpb100_list = list(fpb100_array[:, 1])
print(len(fpb100_list))

453


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

print(device)
torch.cuda.empty_cache()

cuda:0


In [ ]:
# Tokenize the input and put both input and model into GPU
inputs = tokenizer(fpb100_list, padding = True, truncation = True, return_tensors='pt')
print(inputs)

gpu_model = model.to(device)
gpu_inputs = inputs.to(device)


{'input_ids': tensor([[ 101, 4003, 7856,  ...,    0,    0,    0],
        [ 101, 3006, 2951,  ...,    0,    0,    0],
        [ 101, 4654, 2884,  ...,    0,    0,    0],
        ...,
        [ 101, 1996, 3643,  ...,    0,    0,    0],
        [ 101, 6983, 6267,  ...,    0,    0,    0],
        [ 101, 3431, 1999,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [ ]:
# Do the model inference

with torch.no_grad():
  outputs = gpu_model(**gpu_inputs)
print(outputs.logits.shape)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

torch.Size([453, 3])
tensor([[0.0588, 0.1203, 0.8209],
        [0.0253, 0.0231, 0.9516],
        [0.0332, 0.0246, 0.9422],
        ...,
        [0.0807, 0.0114, 0.9079],
        [0.0390, 0.0178, 0.9432],
        [0.0155, 0.9657, 0.0188]], device='cuda:0')


In [ ]:
import pandas as pd

#Headline #Positive #Negative #Neutral
positive = predictions[:, 0].tolist()
negative = predictions[:, 1].tolist()
neutral = predictions[:, 2].tolist()


table = {'Text':fpb100_list,
         "Positive":positive,
         "Negative":negative, 
         "Neutral":neutral}
      
df = pd.DataFrame(table, columns = ["Text", "Positive", "Negative", "Neutral"])

df.head(10)

,Text,Positive,Negative,Neutral
0,"Cardona slowed her vehicle , turned around and...",0.058822,0.120264,0.820914
1,Market data and analytics are derived from pri...,0.025298,0.023113,0.951589
2,Exel is headquartered in Mantyharju in Finland,0.033195,0.024555,0.942250
3,Both operating profit and net sales for the th...,0.958544,0.022159,0.019297
4,Tampere Science Parks is a Finnish company tha...,0.038103,0.013872,0.948025
5,The real estate company posted a net loss of +...,0.013810,0.971294,0.014896
6,The Oxyview Pulse Oximeter is a common device ...,0.048858,0.021268,0.929874
7,Componenta has production lines for similar-si...,0.071035,0.013689,0.915275
8,The EU Commission said earlier it had fined Th...,0.019707,0.953192,0.027101
9,"The company 's scheduled traffic , measured in...",0.954000,0.020504,0.025496


In [ ]:
label_map = {"positive" : 1, "negative" : -1, "neutral" : 0}
print(predictions.shape)

correct = 0.0
for idx, x in enumerate(fpb100_array[:500]):
    true_label = label_map[x[2]]
    pred_list = [float(predictions[idx][0]), float(predictions[idx][1]), float(predictions[idx][2])]
    max_v = max(pred_list)
    if max_v == pred_list[0]:
        prediction_label = 1
    elif max_v == pred_list[1]:
        prediction_label = -1
    else:
        prediction_label = 0
    
    if prediction_label == true_label:
        correct += 1
print("FPB Accurarcy: ", correct / idx * 100, "%")

torch.Size([453, 3])
FPB Accurarcy:  98.45132743362832 %


# Section 2 Adversarially Attack FinBERT

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.models.wrappers import ModelWrapper
import textattack

  
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:36<00:00, 13.1MB/s]
textattack: Unzipping file /root/.cache/textattack/tmp1lmnavnd.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


In [ ]:
# Example of sentiment-classification dataset
import pandas
import numpy as np


fpb100_df = pandas.read_csv("drive/MyDrive/test.csv", sep="\t")

fpb100_array = np.array(fpb100_df)

fpb100_list = []
for idx in range(len(fpb100_array)):
  true_label = 2
  if "positive" == fpb100_array[idx, 2]:
    true_label = 0
  elif "negative" == fpb100_array[idx, 2]:
    true_label = 1
  else:
    true_label = 2
  fpb100_list.append((fpb100_array[idx, 1], true_label))

print(len(fpb100_list))
dataset = textattack.datasets.Dataset(fpb100_list)
for x in dataset:
  print(x)

453
(OrderedDict([('text', 'Cardona slowed her vehicle , turned around and returned to the intersection , where she called 911 ')]), 2)
(OrderedDict([('text', 'Market data and analytics are derived from primary and secondary research ')]), 2)
(OrderedDict([('text', 'Exel is headquartered in Mantyharju in Finland ')]), 2)
(OrderedDict([('text', 'Both operating profit and net sales for the three-month period increased , respectively from EUR16 .0 m and EUR139m , as compared to the corresponding quarter in 2006 ')]), 0)
(OrderedDict([('text', 'Tampere Science Parks is a Finnish company that owns , leases and builds office properties and it specialises in facilities for technology-oriented businesses ')]), 2)
(OrderedDict([('text', 'The real estate company posted a net loss of +ó  x201a -¼ 59.3 million +ó  x201a -¼ 0.21 per share compared with a net profit of +ó  x201a -¼ 31 million +ó  x201a -¼ 0.11 per share for the corresponding quarter of 2007 ')]), 1)
(OrderedDict([('text', 'The Oxyvi

## Section 2.1 Genetic Algorithm
link to the paper: https://arxiv.org/pdf/1804.07998.pdf

In [ ]:
attack = textattack.attack_recipes.genetic_algorithm_alzantot_2018.GeneticAlgorithmAlzantot2018.build(model_wrapper)

textattack: Downloading https://textattack.s3.amazonaws.com/constraints/semantics/language-models/alzantot-goog-lm.
100%|██████████| 3.87G/3.87G [01:39<00:00, 38.8MB/s]
textattack: Unzipping file /root/.cache/textattack/tmp_jhgznks.zip to /root/.cache/textattack/constraints/semantics/language-models/alzantot-goog-lm.
textattack: Successfully saved constraints/semantics/language-models/alzantot-goog-lm to cache.
Recovering graph.


INFO:tensorflow:Recovering Graph /root/.cache/textattack/constraints/semantics/language-models/alzantot-goog-lm/graph-2016-09-10-gpu.pbtxt


Recovering Graph /root/.cache/textattack/constraints/semantics/language-models/alzantot-goog-lm/graph-2016-09-10-gpu.pbtxt
Recovering checkpoint /root/.cache/textattack/constraints/semantics/language-models/alzantot-goog-lm/ckpt-*
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
from textattack import Attacker


attacker = Attacker(attack, dataset)
attacker.attack_dataset()

  0%|          | 0/10 [00:00<?, ?it/s]

Attack(
  (search_method): AlzantotGeneticAlgorithm(
    (pop_size):  60
    (max_iters):  20
    (temp):  0.3
    (give_up_if_no_improvement):  False
    (post_crossover_check):  False
    (max_crossover_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  8
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_percent):  0.2
        (compare_against_original):  True
      )
    (1): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (max_mse_dist):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  False
      )
    (2): GoogleLanguageModel(
        (top_n):  None
        (top_n_per_index):  4
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (

[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:55<08:20, 55.65s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[2 (82%)]] --> [[1 (70%)]]

Cardona slowed her [[vehicle]] , turned [[around]] and [[returned]] to the intersection , where she called 911 

Cardona slowed her [[vehicular]] , turned [[approximately]] and [[revert]] to the intersection , where she called 911 




[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:  20%|██        | 2/10 [04:18<17:13, 129.13s/it]

...

--------------------------------------------- Result 8 ---------------------------------------------
[[2 (92%)]] --> [[[FAILED]]]

Componenta has production lines for similar-sized products at Karkkila in Finland , at Weert in the Netherlands and at Orhangazi in Turkey , and these had a combined output of approximately 100,000 tonnes and net sales of EUR 135 million in 2007 




[Succeeded / Failed / Skipped / Total] 4 / 5 / 0 / 9:  90%|█████████ | 9/10 [50:42<05:38, 338.09s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[[1 (95%)]] --> [[0 (62%)]]

The EU [[Commission]] said earlier it had [[fined]] ThyssenKrupp , United Technologies Corp 's Otis , Schindler AG and Kone Oyj a [[total]] of 992.3 mln eur for alleged cartel activity in the lift market going back twelve years 

The EU [[Committees]] said earlier it had [[fine]] ThyssenKrupp , United Technologies Corp 's Otis , Schindler AG and Kone Oyj a [[generals]] of 992.3 mln eur for alleged cartel activity in the lift market going back twelve years 




[Succeeded / Failed / Skipped / Total] 5 / 5 / 0 / 10: 100%|██████████| 10/10 [52:19<00:00, 313.96s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[0 (95%)]] --> [[1 (53%)]]

The [[company]] 's scheduled traffic , measured in [[revenue]] passenger kilometres RPK , [[grew]] by just over 2 % and nearly 3 % more passengers were carried on scheduled flights than in February 2009 

The [[corporation]] 's scheduled traffic , measured in [[recipes]] passenger kilometres RPK , [[grow]] by just over 2 % and nearly 3 % more passengers were carried on scheduled flights than in February 2009 



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 5      |
| Number of failed attacks:     | 5      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 50.0%  |
| Attack success rate:          | 50.0%  |
| Average perturbed word %:     | 15.04% |
| Average num. words pe

In [ ]:
from textattack import Attacker
from textattack import AttackArgs

attack = textattack.attack_recipes.genetic_algorithm_alzantot_2018.GeneticAlgorithmAlzantot2018.build(model_wrapper)

attack_args = AttackArgs(
    num_examples=50,
)

attacker = Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

textattack: Downloading https://textattack.s3.amazonaws.com/constraints/semantics/language-models/alzantot-goog-lm.
100%|██████████| 3.87G/3.87G [04:12<00:00, 15.3MB/s]
textattack: Unzipping file /root/.cache/textattack/tmp5lfkx_b6.zip to /root/.cache/textattack/constraints/semantics/language-models/alzantot-goog-lm.
textattack: Successfully saved constraints/semantics/language-models/alzantot-goog-lm to cache.
Recovering graph.


INFO:tensorflow:Recovering Graph /root/.cache/textattack/constraints/semantics/language-models/alzantot-goog-lm/graph-2016-09-10-gpu.pbtxt


Recovering Graph /root/.cache/textattack/constraints/semantics/language-models/alzantot-goog-lm/graph-2016-09-10-gpu.pbtxt
Recovering checkpoint /root/.cache/textattack/constraints/semantics/language-models/alzantot-goog-lm/ckpt-*
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
  0%|          | 0/50 [00:00<?, ?it/s]

Attack(
  (search_method): AlzantotGeneticAlgorithm(
    (pop_size):  60
    (max_iters):  20
    (temp):  0.3
    (give_up_if_no_improvement):  False
    (post_crossover_check):  False
    (max_crossover_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  8
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_percent):  0.2
        (compare_against_original):  True
      )
    (1): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (max_mse_dist):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  False
      )
    (2): GoogleLanguageModel(
        (top_n):  None
        (top_n_per_index):  4
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (

[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   2%|▏         | 1/50 [01:01<50:08, 61.39s/it]

...

--------------------------------------------- Result 48 ---------------------------------------------
[[2 (64%)]] --> [[0 (56%)]]

Motorola Inc. of the United States came second with shipments of 217.4 million units for a 21.3 percent market share , [[followed]] by South Korea 's Samsung Electronics Co. with shipments of 118.0 million units for an 11.6 percent share 

Motorola Inc. of the United States came second with shipments of 217.4 million units for a 21.3 percent market share , [[adhered]] by South Korea 's Samsung Electronics Co. with shipments of 118.0 million units for an 11.6 percent share 




[Succeeded / Failed / Skipped / Total] 25 / 24 / 0 / 49:  98%|█████████▊| 49/50 [4:23:49<05:23, 323.05s/it]

--------------------------------------------- Result 49 ---------------------------------------------
[[2 (90%)]] --> [[0 (50%)]]

[[The]] [[donations]] are [[granted]] to Aalto University and the universities of Helsinki , Tampere , Turku , Eastern Finland , Jyv+ñskyl+ñ , Oulu and Vaasa , and to polytechnics to be announced later 

[[Nova]] [[grants]] are [[afforded]] to Aalto University and the universities of Helsinki , Tampere , Turku , Eastern Finland , Jyv+ñskyl+ñ , Oulu and Vaasa , and to polytechnics to be announced later 




[Succeeded / Failed / Skipped / Total] 26 / 24 / 0 / 50: 100%|██████████| 50/50 [4:25:17<00:00, 318.35s/it]

--------------------------------------------- Result 50 ---------------------------------------------
[[2 (65%)]] --> [[0 (60%)]]

Metsa-Botnia will finance the [[payment]] of dividends , the repayment of capital and the repurchase of its own [[shares]] with the [[funds]] deriving from its divestment of the Uruguay operations , and shares in Pohjolan Voima , and by utilising its existing financing facilities 

Metsa-Botnia will finance the [[payout]] of dividends , the repayment of capital and the repurchase of its own [[exchanges]] with the [[resources]] deriving from its divestment of the Uruguay operations , and shares in Pohjolan Voima , and by utilising its existing financing facilities 



+-------------------------------+---------+
| Attack Results                |         |
+-------------------------------+---------+
| Number of successful attacks: | 26      |
| Number of failed attacks:     | 24      |
| Number of skipped attacks:    | 0       |
| Original accuracy:           

## Section 2.2 Faster Alzantot Genetic Algorithm
link to the paper: https://arxiv.org/pdf/1909.00986.pdf

In [ ]:
!pip install torchfile
from textattack import Attacker
from textattack import AttackArgs

attack = textattack.attack_recipes.faster_genetic_algorithm_jia_2019.FasterGeneticAlgorithmJia2019.build(model_wrapper)

attack_args = AttackArgs(
    num_examples=50,
)

attacker = Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5709 sha256=05f62da135f67ca00e2fcfb847ca551648428dfcd9632d0cef14ab0f8fa396ca
  Stored in directory: /root/.cache/pip/wheels/ac/5c/3a/a80e1c65880945c71fd833408cd1e9a8cb7e2f8f37620bb75b
Successfully built torchfile


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.01 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
  0%|          | 0/50 [00:00<?, ?it/s]

Attack(
  (search_method): AlzantotGeneticAlgorithm(
    (pop_size):  60
    (max_iters):  20
    (temp):  0.3
    (give_up_if_no_improvement):  False
    (post_crossover_check):  False
    (max_crossover_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  8
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_percent):  0.2
        (compare_against_original):  True
      )
    (1): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (max_mse_dist):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (2): LearningToWriteLanguageModel(
        (max_log_prob_diff):  5.0
        (compare_against_original):  True
      )
    (3): RepeatModification
    (4): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:   2%|▏         | 1/50 [06:22<5:12:12, 382.30s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[2 (82%)]] --> [[[FAILED]]]

Cardona slowed her vehicle , turned around and returned to the intersection , where she called 911 




...

--------------------------------------------- Result 48 ---------------------------------------------
[[2 (64%)]] --> [[0 (49%)]]

Motorola Inc. of the United States came second with shipments of 217.4 million units for a 21.3 percent market share , [[followed]] by South Korea 's Samsung Electronics Co. with shipments of 118.0 million units for an 11.6 percent share 

Motorola Inc. of the United States came second with shipments of 217.4 million units for a 21.3 percent market share , [[tracked]] by South Korea 's Samsung Electronics Co. with shipments of 118.0 million units for an 11.6 percent share 




[Succeeded / Failed / Skipped / Total] 16 / 33 / 0 / 49:  98%|█████████▊| 49/50 [1:57:04<02:23, 143.36s/it]

--------------------------------------------- Result 49 ---------------------------------------------
[[2 (90%)]] --> [[0 (66%)]]

The [[donations]] are granted to Aalto University and the [[universities]] of Helsinki , Tampere , Turku , Eastern Finland , Jyv+ñskyl+ñ , Oulu and Vaasa , and to polytechnics to be [[announced]] [[later]] 

The [[grants]] are granted to Aalto University and the [[campuses]] of Helsinki , Tampere , Turku , Eastern Finland , Jyv+ñskyl+ñ , Oulu and Vaasa , and to polytechnics to be [[heralded]] [[after]] 




[Succeeded / Failed / Skipped / Total] 17 / 33 / 0 / 50: 100%|██████████| 50/50 [1:57:19<00:00, 140.78s/it]

--------------------------------------------- Result 50 ---------------------------------------------
[[2 (65%)]] --> [[0 (55%)]]

Metsa-Botnia will finance the [[payment]] of dividends , the repayment of capital and the repurchase of its own [[shares]] with the funds deriving from its divestment of the Uruguay operations , and shares in Pohjolan Voima , and by utilising its existing financing facilities 

Metsa-Botnia will finance the [[payout]] of dividends , the repayment of capital and the repurchase of its own [[interchange]] with the funds deriving from its divestment of the Uruguay operations , and shares in Pohjolan Voima , and by utilising its existing financing facilities 



+-------------------------------+---------+
| Attack Results                |         |
+-------------------------------+---------+
| Number of successful attacks: | 17      |
| Number of failed attacks:     | 33      |
| Number of skipped attacks:    | 0       |
| Original accuracy:            | 100.0% 

## Section 2.3 Improved Genetic Algorithm
link to the paper: https://arxiv.org/pdf/1909.06723.pdf

In [ ]:
from textattack import Attacker
from textattack import AttackArgs

attack = textattack.attack_recipes.iga_wang_2019.IGAWang2019.build(model_wrapper)

attack_args = AttackArgs(
    num_examples=50,
)

attacker = Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
  0%|          | 0/50 [00:00<?, ?it/s]

Attack(
  (search_method): ImprovedGeneticAlgorithm(
    (pop_size):  60
    (max_iters):  20
    (temp):  0.3
    (give_up_if_no_improvement):  False
    (post_crossover_check):  False
    (max_crossover_retries):  20
    (max_replace_times_per_index):  5
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_percent):  0.2
        (compare_against_original):  True
      )
    (1): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (max_mse_dist):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  False
      )
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   2%|▏         | 1/50 [00:04<03:19,  4.07s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[2 (82%)]] --> [[1 (58%)]]

Cardona slowed her vehicle , turned [[around]] and [[returned]] to the intersection , where she called 911 

Cardona slowed her vehicle , turned [[roughly]] and [[reverting]] to the intersection , where she called 911 




[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:   4%|▍         | 2/50 [00:40<16:22, 20.46s/it]

...

--------------------------------------------- Result 48 ---------------------------------------------
[[2 (64%)]] --> [[0 (56%)]]

Motorola Inc. of the United States came second with shipments of 217.4 million units for a 21.3 percent market share , [[followed]] by South Korea 's Samsung Electronics Co. with shipments of 118.0 million units for an 11.6 percent share 

Motorola Inc. of the United States came second with shipments of 217.4 million units for a 21.3 percent market share , [[adhered]] by South Korea 's Samsung Electronics Co. with shipments of 118.0 million units for an 11.6 percent share 




[Succeeded / Failed / Skipped / Total] 42 / 7 / 0 / 49:  98%|█████████▊| 49/50 [29:25<00:36, 36.04s/it]

--------------------------------------------- Result 49 ---------------------------------------------
[[2 (90%)]] --> [[0 (72%)]]

The donations are granted to Aalto University and the universities of Helsinki , Tampere , Turku , Eastern Finland , Jyv+ñskyl+ñ , Oulu and Vaasa , and to polytechnics to be [[announced]] [[later]] 

The donations are granted to Aalto University and the universities of Helsinki , Tampere , Turku , Eastern Finland , Jyv+ñskyl+ñ , Oulu and Vaasa , and to polytechnics to be [[heralding]] [[following]] 




[Succeeded / Failed / Skipped / Total] 43 / 7 / 0 / 50: 100%|██████████| 50/50 [29:34<00:00, 35.49s/it]

--------------------------------------------- Result 50 ---------------------------------------------
[[2 (65%)]] --> [[0 (57%)]]

Metsa-Botnia will finance the [[payment]] of dividends , the repayment of capital and the repurchase of its own shares with the [[funds]] deriving from its divestment of the Uruguay operations , and shares in Pohjolan Voima , and by utilising its existing financing facilities 

Metsa-Botnia will finance the [[payout]] of dividends , the repayment of capital and the repurchase of its own shares with the [[resources]] deriving from its divestment of the Uruguay operations , and shares in Pohjolan Voima , and by utilising its existing financing facilities 



+-------------------------------+---------+
| Attack Results                |         |
+-------------------------------+---------+
| Number of successful attacks: | 43      |
| Number of failed attacks:     | 7       |
| Number of skipped attacks:    | 0       |
| Original accuracy:            | 100.0%  

In [ ]:
from textattack import Attacker
from textattack import AttackArgs

attack = textattack.attack_recipes.iga_wang_2019.IGAWang2019.build(model_wrapper)

attack_args = AttackArgs(
    num_examples=-1,
)

attacker = Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
  0%|          | 0/453 [00:00<?, ?it/s]

Attack(
  (search_method): ImprovedGeneticAlgorithm(
    (pop_size):  60
    (max_iters):  20
    (temp):  0.3
    (give_up_if_no_improvement):  False
    (post_crossover_check):  False
    (max_crossover_retries):  20
    (max_replace_times_per_index):  5
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_percent):  0.2
        (compare_against_original):  True
      )
    (1): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (max_mse_dist):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  False
      )
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|          | 1/453 [00:04<32:42,  4.34s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[2 (82%)]] --> [[1 (58%)]]

Cardona slowed her vehicle , turned [[around]] and [[returned]] to the intersection , where she called 911 

Cardona slowed her vehicle , turned [[roughly]] and [[reverting]] to the intersection , where she called 911 




[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:   0%|          | 2/453 [00:41<2:35:24, 20.68s/it]

...

--------------------------------------------- Result 451 ---------------------------------------------
[[2 (91%)]] --> [[[FAILED]]]

The value of the orders is over EUR 10mn 




[Succeeded / Failed / Skipped / Total] 315 / 129 / 8 / 452: 100%|█████████▉| 452/453 [6:02:56<00:48, 48.18s/it]

--------------------------------------------- Result 452 ---------------------------------------------
[[2 (94%)]] --> [[0 (51%)]]

Finnish printed circuit boards ( PCBs ) [[maker]] Aspocomp [[Group]] Oyj [[said]] on December 4 , 2006 it named [[Henry]] Gilchrist [[senior]] [[vice]] president of the group 's Asian operations , as of [[January]] 8 , 2007 

Finnish printed circuit boards ( PCBs ) [[producers]] Aspocomp [[Grupo]] Oyj [[affirms]] on December 4 , 2006 it named [[Enrique]] Gilchrist [[top]] [[immodest]] president of the group 's Asian operations , as of [[Janeiro]] 8 , 2007 




[Succeeded / Failed / Skipped / Total] 316 / 129 / 8 / 453: 100%|██████████| 453/453 [6:03:00<00:00, 48.08s/it]

--------------------------------------------- Result 453 ---------------------------------------------
[[1 (97%)]] --> [[0 (94%)]]

Changes in the market situation and tougher price competition have substantially [[reduced]] demand for bread packaging manufactured at the Kauhava plant , according to the company 

Changes in the market situation and tougher price competition have substantially [[mitigation]] demand for bread packaging manufactured at the Kauhava plant , according to the company 



+-------------------------------+---------+
| Attack Results                |         |
+-------------------------------+---------+
| Number of successful attacks: | 316     |
| Number of failed attacks:     | 129     |
| Number of skipped attacks:    | 8       |
| Original accuracy:            | 98.23%  |
| Accuracy under attack:        | 28.48%  |
| Attack success rate:          | 71.01%  |
| Average perturbed word %:     | 15.98%  |
| Average num. words per input: | 20.95   |
| Avg num que

## Section 2.4 Visual Textbugger Attack
link to paper: https://arxiv.org/abs/1812.05271

In [ ]:
from textattack import Attacker
from textattack import AttackArgs

attack = textattack.attack_recipes.textbugger_li_2018.TextBuggerLi2018.build(model_wrapper)
# attack = textattack.attack_recipes.morpheus_tan_2020.MorpheusTan2020.build(model_wrapper)

attack_args = AttackArgs(
    num_examples=-1,
)

attacker = Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.



  0%|          | 0/453 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (2): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (3): WordSwapHomoglyphSwap
    (4): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



  0%|          | 0/453 [04:10<?, ?it/s]



  0%|          | 1/453 [00:17<2:09:34, 17.20s/it]


[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|          | 1/453 [00:17<2:09:36, 17.20s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[2 (82%)]] --> [[1 (84%)]]

Cardona slowed her vehicle , turned around and returned to the intersection , where she [[called]] 911 

Cardona slowed her vehicle , turned around and returned to the intersection , where she [[claled]] 911 







[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|          | 2/453 [00:17<1:07:23,  8.97s/it]


[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:   0%|          | 2/453 [00:17<1:07:24,  8.97s/it]

...

--------------------------------------------- Result 451 ---------------------------------------------
[[2 (91%)]] --> [[[FAILED]]]

The value of the orders is over EUR 10mn 







[Succeeded / Failed / Skipped / Total] 185 / 258 / 8 / 451: 100%|█████████▉| 452/453 [11:01<00:01,  1.46s/it]


[Succeeded / Failed / Skipped / Total] 185 / 259 / 8 / 452: 100%|█████████▉| 452/453 [11:01<00:01,  1.46s/it]

--------------------------------------------- Result 452 ---------------------------------------------
[[2 (94%)]] --> [[[FAILED]]]

Finnish printed circuit boards ( PCBs ) maker Aspocomp Group Oyj said on December 4 , 2006 it named Henry Gilchrist senior vice president of the group 's Asian operations , as of January 8 , 2007 







[Succeeded / Failed / Skipped / Total] 186 / 259 / 8 / 453: 100%|██████████| 453/453 [11:02<00:00,  1.46s/it]

--------------------------------------------- Result 453 ---------------------------------------------
[[1 (97%)]] --> [[0 (62%)]]

Changes in the market situation and tougher price competition have substantially reduced [[demand]] for bread packaging manufactured at the Kauhava plant , according to the company 

Changes in the market situation and tougher price competition have substantially reduced [[demands]] for bread packaging manufactured at the Kauhava plant , according to the company 



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 186    |
| Number of failed attacks:     | 259    |
| Number of skipped attacks:    | 8      |
| Original accuracy:            | 98.23% |
| Accuracy under attack:        | 57.17% |
| Attack success rate:          | 41.8%  |
| Average perturbed word %:     | 20.07% |
| Average num. words per input: | 20.95  |
| Avg num queries:        